## Bidirectional LSTM(Long Short Term memory) model for heart failure prediction

### Dataset

https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.init as init

import torchmetrics

import os

In [2]:
num_cores_os = os.cpu_count()
print(f"Number of available CPUs: {num_cores_os}")

# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    # Get the number of available GPUs and store it in a variable
    num_gpus = torch.cuda.device_count()
else:
    num_gpus = 0  # No GPUs available

print(f"Number of available GPUs: {num_gpus}")

Number of available CPUs: 8
Number of available GPUs: 0


# Preparing the dataset for training

In [3]:
# Load your dataset, assuming it's in a CSV file
df = pd.read_csv('heart.xls')

print(df)

     Age Sex ChestPainType  RestingBP  Cholesterol  FastingBS RestingECG  \
0     40   M           ATA        140          289          0     Normal   
1     49   F           NAP        160          180          0     Normal   
2     37   M           ATA        130          283          0         ST   
3     48   F           ASY        138          214          0     Normal   
4     54   M           NAP        150          195          0     Normal   
..   ...  ..           ...        ...          ...        ...        ...   
913   45   M            TA        110          264          0     Normal   
914   68   M           ASY        144          193          1     Normal   
915   57   M           ASY        130          131          0     Normal   
916   57   F           ATA        130          236          0        LVH   
917   38   M           NAP        138          175          0     Normal   

     MaxHR ExerciseAngina  Oldpeak ST_Slope  HeartDisease  
0      172              N  

In [4]:
# Encode categorical variables
encoder = LabelEncoder()
categorical_cols = ["Sex", "ChestPainType", "RestingECG", "ExerciseAngina", "ST_Slope"]
for col in categorical_cols:
    df[col] = encoder.fit_transform(df[col])


print(df)

     Age  Sex  ChestPainType  RestingBP  Cholesterol  FastingBS  RestingECG  \
0     40    1              1        140          289          0           1   
1     49    0              2        160          180          0           1   
2     37    1              1        130          283          0           2   
3     48    0              0        138          214          0           1   
4     54    1              2        150          195          0           1   
..   ...  ...            ...        ...          ...        ...         ...   
913   45    1              3        110          264          0           1   
914   68    1              0        144          193          1           1   
915   57    1              0        130          131          0           1   
916   57    0              1        130          236          0           0   
917   38    1              2        138          175          0           1   

     MaxHR  ExerciseAngina  Oldpeak  ST_Slope  Hear

In [5]:
features = df.drop('HeartDisease', axis=1).values
target = df['HeartDisease'].values
print(features)

[[40.   1.   1.  ...  0.   0.   2. ]
 [49.   0.   2.  ...  0.   1.   1. ]
 [37.   1.   1.  ...  0.   0.   2. ]
 ...
 [57.   1.   0.  ...  1.   1.2  1. ]
 [57.   0.   1.  ...  0.   0.   1. ]
 [38.   1.   2.  ...  0.   0.   2. ]]


# Normalize the dataset for faster training

In [6]:
scaler = StandardScaler()

features = scaler.fit_transform(features)

print(features)

[[-1.4331398   0.51595242  0.22903206 ... -0.8235563  -0.83243239
   1.05211381]
 [-0.47848359 -1.93816322  1.27505906 ... -0.8235563   0.10566353
  -0.59607813]
 [-1.75135854  0.51595242  0.22903206 ... -0.8235563  -0.83243239
   1.05211381]
 ...
 [ 0.37009972  0.51595242 -0.81699495 ...  1.21424608  0.29328271
  -0.59607813]
 [ 0.37009972 -1.93816322  0.22903206 ... -0.8235563  -0.83243239
  -0.59607813]
 [-1.64528563  0.51595242  1.27505906 ... -0.8235563  -0.83243239
   1.05211381]]


In [7]:
# Convert features and target to NumPy arrays
X = np.array(features, dtype=np.float32)
y = np.array(target, dtype=np.float32)

In [8]:
X = np.reshape(X, (918, 1, 11))

print(X)
print(X.shape)

[[[-1.4331398   0.5159524   0.22903205 ... -0.8235563  -0.8324324
    1.0521138 ]]

 [[-0.4784836  -1.9381633   1.2750591  ... -0.8235563   0.10566353
   -0.59607816]]

 [[-1.7513585   0.5159524   0.22903205 ... -0.8235563  -0.8324324
    1.0521138 ]]

 ...

 [[ 0.37009972  0.5159524  -0.81699497 ...  1.214246    0.29328272
   -0.59607816]]

 [[ 0.37009972 -1.9381633   0.22903205 ... -0.8235563  -0.8324324
   -0.59607816]]

 [[-1.6452856   0.5159524   1.2750591  ... -0.8235563  -0.8324324
    1.0521138 ]]]
(918, 1, 11)


In [9]:
y = np.reshape(y, (918, 1))

print(y.shape)

(918, 1)


# Split the dataset into training developement and test sets

In [10]:
X_train, X_dev_and_test, y_train, y_dev_and_test = train_test_split(X, y, test_size=0.4, random_state=42)

X_dev, X_test, y_dev, y_test = train_test_split(X_dev_and_test, y_dev_and_test, test_size=0.5, random_state=42)

In [11]:
# Convert NumPy arrays to PyTorch tensors
X_train = torch.tensor(X_train).clone().detach()
y_train = torch.tensor(y_train).clone().detach()

X_dev = torch.tensor(X_dev).clone().detach()
y_dev = torch.tensor(y_dev).clone().detach()


X_test = torch.tensor(X_test).clone().detach()
y_test = torch.tensor(y_test).clone().detach()

print(X_train.shape)
print(y_train.shape)

print(X_dev.shape)
print(y_dev.shape)

print(X_test.shape)
print(y_test.shape)

torch.Size([550, 1, 11])
torch.Size([550, 1])
torch.Size([184, 1, 11])
torch.Size([184, 1])
torch.Size([184, 1, 11])
torch.Size([184, 1])


# Setting the model hyperparameters

In [12]:
batch_size=64
learning_rate=0.001
hidden_dim=32
num_layers=5
num_epochs = 30

input_dim=11
output_dim=1

first_fc_layer = 100
second_fc_layer = 20
 
lstm_dropout = 0.4
dropout_prob = 0.3

num_classes=2

features = 11

# The complete model

In [13]:
class HeartPredictionModel(pl.LightningModule):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(HeartPredictionModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True, dropout=lstm_dropout)
        #init.xavier_uniform_(self.lstm.weight)
                
        self.fc1 = nn.Linear(2* hidden_dim, first_fc_layer) 
        init.xavier_uniform_(self.fc1.weight)
        self.dropout = nn.Dropout(dropout_prob)  # Add dropout layer
        
        self.fc2 = nn.Linear(first_fc_layer, second_fc_layer) 
        init.xavier_uniform_(self.fc2.weight)
        self.dropout = nn.Dropout(dropout_prob)  # Add dropout layer

        self.fc3 = nn.Linear(second_fc_layer, output_dim) 
        init.xavier_uniform_(self.fc3.weight)
        
        self.sigmoid = nn.Sigmoid()  # Add a sigmoid activation function
        
    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]  # Get the last output from the sequence for each batch
        out = self.dropout(out)  # Apply dropout
        out = self.fc1(out)
        out = self.dropout(out)  # Apply dropout
        out = self.fc2(out)
        out = self.dropout(out)  # Apply dropout
        out = self.fc3(out)
        out = self.sigmoid(out)  # Apply sigmoid activation

        return out 

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)
        scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
        return [optimizer], [scheduler]
        
    def training_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self(inputs)
        loss = nn.BCELoss()(outputs, targets)
        self.log('train_loss', loss, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self(inputs)
        loss = nn.BCELoss()(outputs, targets)
        self.log('val_loss', loss, on_step=False, on_epoch=True)
        return loss

    def train_dataloader(self):
        dataset = TensorDataset(X_train, y_train)
        return DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=num_cores_os)

    def val_dataloader(self):
        dataset = TensorDataset(X_dev, y_dev)
        return DataLoader(dataset, batch_size=batch_size, drop_last=True, num_workers=num_cores_os)

In [14]:
model = HeartPredictionModel(input_dim=input_dim, hidden_dim=hidden_dim, num_layers=num_layers, output_dim=output_dim)

# Initialize a PyTorch Lightning trainer
trainer = pl.Trainer(max_epochs=num_epochs)  # Set the number of epochs and GPU usage as needed

# Train the model
trainer.fit(model)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\tiago\Projects\deepLearning\envi\Lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(

  | Name    | Type    | Params
------------------------------------
0 | lstm    | LSTM    | 111 K 
1 | fc1     | Linear  | 6.5 K 
2 | dropout | Dropout | 0     
3 | fc2     | Linear  | 2.0 K 
4 | fc3     | Linear  | 21    
5 | sigmoid | Sigmoid | 0     
-------------------

Sanity Checking: 0it [00:00, ?it/s]

C:\Users\tiago\Projects\deepLearning\envi\Lib\site-packages\pytorch_lightning\loops\fit_loop.py:281: PossibleUserWarning: The number of training batches (8) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=30` reached.
